### Extracting and Saving keypoints from LRO WAC Mosaic.

In [ ]:
import cv2
import numpy as np
from osgeo import gdal

# Open the .tif file using GDAL
lro_wac_path = '/content/drive/MyDrive/Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013.tif'
dataset = gdal.Open(lro_wac_path)

# Get image dimensions
img_width = dataset.RasterXSize
img_height = dataset.RasterYSize

# Define tile size (adjust based on memory constraints)
tile_size = 1000  # Example tile size
overlap = 100     # Overlap to ensure no features are missed between tiles

# Initialize SIFT
sift = cv2.SIFT_create()

# Loop over the image in tiles
keypoints_wac = []
descriptors_wac = []

for y in range(0, img_height, tile_size - overlap):
    for x in range(0, img_width, tile_size - overlap):
        # Calculate the bounds of the tile
        x_end = min(x + tile_size, img_width)
        y_end = min(y + tile_size, img_height)

        # Read the tile from the image
        tile = dataset.ReadAsArray(x, y, x_end - x, y_end - y)

        # If the image has multiple bands, select the first one (grayscale)
        if tile.ndim > 2:
            tile = tile[0]

        # Convert to 8-bit image if needed
        tile = cv2.normalize(tile, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

        # Detect keypoints and compute descriptors for the tile
        kp, des = sift.detectAndCompute(tile, None)
        keypoints_wac.extend(kp)
        if des is not None:
            descriptors_wac.append(des)

# Concatenate all descriptors into a single array
if descriptors_wac:
    descriptors_wac = np.vstack(descriptors_wac)

print(f"Extracted {len(keypoints_wac)} keypoints from the LRO WAC mosaic in tiles.")

### Loading mosaic features

In [ ]:
import cv2
import pickle
# Load the previously saved features of the LRO WAC mosaic
with open('/kaggle/input/bah-2024/lro_wac_features.pkl', 'rb') as f:
    keypoints_wac_serialized, descriptors_wac = pickle.load(f)

#### Converting from serialized to keypoints

In [ ]:
import cv2
import pickle

# Function to convert serialized keypoints back to cv2.KeyPoint format
def convert_serializable_to_keypoints(serializable_keypoints):
    return [
        cv2.KeyPoint(
            x=kp[0][0],  # x-coordinate
            y=kp[0][1],  # y-coordinate
            size=kp[1],   # size
            angle=kp[2],  # angle
            response=kp[3],  # response
            octave=kp[4],    # octave
            class_id=kp[5]   # class_id
        )
        for kp in serializable_keypoints
    ]

# Load the serialized keypoints and descriptors from the file
with open('/kaggle/input/bah-2024/lro_wac_features.pkl', 'rb') as f:
    keypoints_wac_serializable, descriptors_wac = pickle.load(f)

# Convert the keypoints back to cv2.KeyPoint format
keypoints_wac = convert_serializable_to_keypoints(keypoints_wac_serializable)

print("Features loaded and keypoints restored.")

#### Loading TMC image (downsampled and with reduced bit structure)

In [ ]:
import cv2

# Load the .tif image
image_path = '/kaggle/input/bah-2024/converted.tif'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Initialize the SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints, descriptors = sift.detectAndCompute(image, None)

# Store keypoints in a variable
keypoints_list = [(kp.pt, kp.size, kp.angle, kp.response, kp.octave) for kp in keypoints]

# Optionally, you can visualize keypoints on the image
output_image = cv2.drawKeypoints(image, keypoints, None)

# Save the output image to verify the keypoints
cv2.imwrite('keypoints_output.tif', output_image)

print(f"Number of keypoints detected: {len(keypoints)}")
print(f"Keypoints list: {keypoints_list}")

#### Extracting keypoints of TMC image

In [ ]:
import numpy as np
import cv2

# Ensure descriptors are continuous and of type float32
descriptors_tmc_as_array = np.ascontiguousarray(descriptors, dtype=np.float32)
descriptors_wac_as_array = np.ascontiguousarray(descriptors, dtype=np.float32)

# Check the shapes of the descriptors
print(f"Shape of descriptors_tmc_as_array: {descriptors_tmc_as_array.shape}")
print(f"Shape of descriptors_wac_as_array: {descriptors_wac_as_array.shape}")

# Set up FLANN-based matcher
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

# Process the WAC descriptors in smaller batches
batch_size = 100000  # Adjust based on your system's memory capacity
good_matches = []

for i in range(0, len(descriptors_wac_as_array), batch_size):
    batch_descriptors_wac = descriptors_wac_as_array[i:i + batch_size]

    try:
        matches = flann.knnMatch(descriptors_tmc_as_array, batch_descriptors_wac, k=2)
    except cv2.error as e:
        print(f"Error during knnMatch with batch {i}-{i+batch_size}: {e}")
        continue

    # Apply Lowe's ratio test to filter good matches
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

print(f"Found {len(good_matches)} good matches.")

#### Matching TMC image with extracted keypoints